# Imports and definitions

In [1]:
import pickle
from pathlib import Path

import polars as pl
import polars.selectors as cs

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import plotly.express as px


_ = pl.Config.set_tbl_cols(None)
_ = pl.Config.set_fmt_str_lengths(500)
_ = pl.Config.set_fmt_float("full")

In [2]:
base_dir = Path('/workspaces/data-scientist-at-magenta/')
code_dir = base_dir / 'notebooks'
dagster_storage_dir = base_dir / 'dagster_home' / 'storage'

In [3]:
def load_artifact(targ_file:str):
    targ_path = dagster_storage_dir / targ_file
    
    if not targ_path.exists():
        raise FileNotFoundError(f'Artifact {targ_file} not found in {dagster_storage_dir}')

    with open(targ_path,'rb') as fp:
        test_artifact = pickle.load(fp)

    return pl.from_pandas(test_artifact)

`core_data` <br><br>

| Feature Name           | Description                                                  |
|------------------------|--------------------------------------------------------------|
| rating_account_id      | Unique identifier for the contract account                    |
| customer_id            | Unique identifier for the customer                           |
| age                    | Age of the customer **in years**                                       |
| contract_lifetime_days | Total duration of the customer contract in days              |
| remaining_binding_days | Number of days left in the contract binding period - usual binding period is 2 years - **if it's positive it means that the customer is still in the binding period**       |
| has_special_offer      | Indicates if the customer has a special offer      |
| is_magenta1_customer   | Indicates if the customer is part of the Magenta1 program - fedelty program    |
| available_gb           | Amount of mobile data included in the current tariff         |
| gross_mrc              | Gross monthly recurring charge (in euros)                    |
| smartphone_brand       | Brand of the customer’s smartphone                           |
| has_done_upselling     | Whether the customer has already done an upsell in the last 3 years      |


`usage_info`

| Feature Name           | Description                                                  |
|------------------------|--------------------------------------------------------------|
| rating_account_id      | Unique identifier for the contract account                    |
| billed_period_month_d  | Billing period (monthly)                                     |
| has_used_roaming       | Indicates if roaming was used during the period            |
| used_gb                | Amount of mobile data used in the billing period (in GB)     |


`customer_interactions`

| Feature Name   | Description                                                              |
|----------------|--------------------------------------------------------------------------|
| customer_id    | Unique identifier for the customer                                       |
| type_subtype   | Category and subtype of the interaction (e.g., tariff change, billing)   |
| n              | Number of interactions of this type in the last 6 months                                |
| days_since_last| Number of days since the last interaction of this type                   |


# Read data

In [4]:
%%time

core_data = load_artifact('core_data')
customer_interactions = load_artifact('customer_interactions')
usage_info = load_artifact('usage_info')

CPU times: user 119 ms, sys: 31.1 ms, total: 150 ms
Wall time: 197 ms


---

# Features computation

## `core_data`

In [5]:
%%time

core_data = core_data.with_columns(
    pl.col('rating_account_id').cast(pl.Utf8),
    pl.col("has_done_upselling").cast(pl.Boolean),
    pl.col("has_special_offer").cast(pl.Boolean),
    pl.col("is_magenta1_customer").cast(pl.Boolean)
)

# Manipulating binding days
core_data = core_data.with_columns(
    (pl.col('contract_lifetime_days') + pl.col('remaining_binding_days')).alias('contract_binding_days'),
    (pl.col('contract_lifetime_days') / (pl.col('contract_lifetime_days') + pl.col('remaining_binding_days'))).round(2).alias('completion_rate'),
    pl.when(pl.col('remaining_binding_days') > 0)
        .then(True)
        .otherwise(False)
        .alias('is_bounded')
)


# One-hot-encoding smartphone brands - extracting the values in order to keep the same order for the columns
# The number of unique values is not too high, so one-hot-encoding is not affecting the dimensionality too much
smartphone_brands_list = core_data.select(pl.col('smartphone_brand')).unique().to_series().sort().to_list()
core_data = core_data.with_columns(
    [
        pl.when(pl.col("smartphone_brand") == brand)
        .then(True)
        .otherwise(False)
        .alias(f"is_{brand.lower()}")
        for brand in smartphone_brands_list
    ]
)
core_data = core_data.drop("smartphone_brand")


# Add how many contract has the customer - including the current one
n_contract_per_customer = core_data.group_by("customer_id").agg(
    pl.col("rating_account_id").count().alias("n_contracts_per_customer")
)
core_data = core_data.join(n_contract_per_customer, on="customer_id", how="left")

CPU times: user 38.1 ms, sys: 8.92 ms, total: 47 ms
Wall time: 61.9 ms


In [6]:
core_data

rating_account_id,customer_id,age,contract_lifetime_days,remaining_binding_days,has_special_offer,is_magenta1_customer,available_gb,gross_mrc,has_done_upselling,contract_binding_days,completion_rate,is_bounded,is_huawei,is_oneplus,is_samsung,is_xiaomi,is_iphone,n_contracts_per_customer
str,str,i64,i64,i64,bool,bool,i64,f64,bool,i64,f64,bool,bool,bool,bool,bool,bool,u32
"""289094""","""4.161115""",36,878,325,false,false,20,70,false,1203,0.73,true,false,false,false,false,true,1
"""677626""","""2.429976""",34,998,614,false,false,0,5,false,1612,0.62,true,false,false,true,false,false,1
"""769928""","""3.875044""",36,37,-26,false,true,50,16.94,false,11,3.36,false,false,false,true,false,false,2
"""873260""","""4.649933""",50,503,-149,false,true,20,30.2,true,354,1.42,false,false,false,false,false,true,1
"""109774""","""3.851059""",47,331,-328,true,true,null,46.12,false,3,110.33,false,false,false,true,false,false,3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""502283""","""5.605022""",88,1573,-576,false,false,10,34.18,false,997,1.58,false,false,false,true,false,false,4
"""618421""","""2.862063""",85,1138,412,true,false,40,50.1,false,1550,0.73,true,false,false,false,false,true,1
"""104422""","""2.414264""",79,1709,-494,false,false,10,12.96,false,1215,1.41,false,false,false,true,false,false,3


In [7]:
core_data.shape

(100000, 19)

---

## `usage_info`

In [8]:
%%time

usage_info = usage_info.with_columns([
    pl.col('rating_account_id').cast(pl.Utf8),
    pl.col('billed_period_month_d').cast(pl.Date),
    pl.col('has_used_roaming').cast(pl.Boolean),
    pl.col('used_gb').cast(pl.Float64)
]).sort(['rating_account_id', 'billed_period_month_d'])

CPU times: user 219 ms, sys: 6.43 ms, total: 226 ms
Wall time: 196 ms


In [9]:
%%time

month_usage = usage_info.group_by('rating_account_id').agg([
    pl.col('used_gb')
])

month_usage = month_usage.with_columns([
    pl.col('used_gb').list.get(0).alias('last_1_month_usage_gb'),
    pl.col('used_gb').list.get(1).alias('last_2_month_usage_gb'),
    pl.col('used_gb').list.get(2).alias('last_3_month_usage_gb'),
    pl.col('used_gb').list.get(3).alias('last_4_month_usage_gb'),

]).drop('used_gb')


CPU times: user 7.22 ms, sys: 1.73 ms, total: 8.95 ms
Wall time: 7.52 ms


In [10]:
month_usage

rating_account_id,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb
str,f64,f64,f64,f64
"""100010""",0.9,0.8,0.4,0.9
"""100017""",0.7,0.5,0.8,0.3
"""100036""",0.9,0.2,0.2,1
"""100047""",35.6,48.9,35.4,50.7
"""100064""",0.7,0.7,0.3,0.8
…,…,…,…,…
"""999922""",2.3,1.4,3.6,2.1
"""999934""",12.8,13.8,14.5,6.5
"""999940""",9.8,11.6,5.4,14.4


In [11]:
%%time

aggregated_features = usage_info.group_by('rating_account_id').agg([
    
    # BASIC USAGE STATISTICS
    pl.col('used_gb').mean().round(2).alias('avg_monthly_usage_gb'),
    pl.col('used_gb').median().round(2).alias('median_monthly_usage_gb'),
    pl.col('used_gb').sum().round(2).alias('total_usage_gb'),
    pl.col('used_gb').std().round(2).alias('usage_std_gb'),
    pl.col('used_gb').min().round(2).alias('min_monthly_usage_gb'),
    pl.col('used_gb').max().round(2).alias('max_monthly_usage_gb'),
    pl.col('used_gb').quantile(0.25).round(2).alias('usage_q25_gb'),
    pl.col('used_gb').quantile(0.75).round(2).alias('usage_q75_gb'),
    
    # ROAMING STATISTICS
    pl.col('has_used_roaming').sum().alias('months_with_roaming'),
    pl.col('has_used_roaming').any().alias('ever_used_roaming'),
    pl.col('has_used_roaming').all().alias('always_used_roaming'),
    
    # # USAGE INTENSITY CATEGORIES
    (pl.col('used_gb') == 0).sum().alias('zero_usage_months'),
    (pl.col('used_gb') > 0).sum().alias('active_usage_months'),
])

CPU times: user 377 ms, sys: 15.3 ms, total: 393 ms
Wall time: 330 ms


In [12]:
aggregated_features

rating_account_id,avg_monthly_usage_gb,median_monthly_usage_gb,total_usage_gb,usage_std_gb,min_monthly_usage_gb,max_monthly_usage_gb,usage_q25_gb,usage_q75_gb,months_with_roaming,ever_used_roaming,always_used_roaming,zero_usage_months,active_usage_months
str,f64,f64,f64,f64,f64,f64,f64,f64,u32,bool,bool,u32,u32
"""100010""",0.75,0.85,3,0.24,0.4,0.9,0.8,0.9,0,false,false,0,4
"""100017""",0.57,0.6,2.3,0.22,0.3,0.8,0.5,0.7,2,true,false,0,4
"""100036""",0.57,0.55,2.3,0.43,0.2,1,0.2,0.9,1,true,false,0,4
"""100047""",42.65,42.25,170.6,8.29,35.4,50.7,35.6,48.9,0,false,false,0,4
"""100064""",0.62,0.7,2.5,0.22,0.3,0.8,0.7,0.7,1,true,false,0,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""999922""",2.35,2.2,9.4,0.92,1.4,3.6,2.1,2.3,0,false,false,0,4
"""999934""",11.9,13.3,47.6,3.67,6.5,14.5,12.8,13.8,3,true,false,0,4
"""999940""",10.3,10.7,41.2,3.78,5.4,14.4,9.8,11.6,1,true,false,0,4


In [13]:
%%time

# CALCULATE TRENDS AND ROLLING METRICS
trend_features = usage_info.group_by('rating_account_id').agg([
    # ROLLING AVERAGES
    # 2-month rolling average
    pl.col('used_gb').rolling_mean_by(
        'billed_period_month_d', window_size='2mo'
    ).alias('avg_2month_rolling_usage_gb'),
    
    # 3-month rolling average
        pl.col('used_gb').rolling_mean_by(
        'billed_period_month_d', window_size='3mo'
    ).alias('avg_3month_rolling_usage_gb'),

    
    # PERIOD-OVER-PERIOD DELTAS
    (pl.col('used_gb') - pl.col('used_gb').shift(1)).alias('delta_1mo'),
    (pl.col('used_gb') - pl.col('used_gb').shift(2)).alias('delta_2mo'),
    (pl.col('used_gb') - pl.col('used_gb').shift(3)).alias('delta_3mo'),

    # VOLATILITY METRICS
    # Rolling standard deviation
    pl.col('used_gb').rolling_std_by(
        'billed_period_month_d', window_size='2mo'
    ).alias('std_2month_rolling_usage_gb')
])

trend_features = trend_features.with_columns([
    # delta_1mo statistics
    pl.col('delta_1mo').list.mean().round(2).alias('avg_delta_1mo'),
    pl.col('delta_1mo').list.std().round(2).alias('delta_1mo_volatility'),
    pl.col('delta_1mo').list.max().round(2).alias('max_delta_1mo_increase'),
    pl.col('delta_1mo').list.min().round(2).alias('max_delta_1mo_decrease'),
    pl.col('delta_1mo').list.eval(pl.element() > 0).list.sum().alias('months_with_delta_1mo_increase'),
    pl.col('delta_1mo').list.eval(pl.element() < 0).list.sum().alias('months_with_delta_1mo_decrease'),
    pl.col('delta_1mo').list.eval(pl.element() == 0).list.sum().alias('months_with_no_delta_1mo_change'),

    # delta_2mo statistics
    pl.col('delta_2mo').list.mean().round(2).alias('avg_delta_2mo'),
    pl.col('delta_2mo').list.std().round(2).alias('delta_2mo_volatility'),
    pl.col('delta_2mo').list.max().round(2).alias('max_delta_2mo_increase'),
    pl.col('delta_2mo').list.min().round(2).alias('max_delta_2mo_decrease'),
    pl.col('delta_2mo').list.eval(pl.element() > 0).list.sum().alias('months_with_delta_2mo_increase'),
    pl.col('delta_2mo').list.eval(pl.element() < 0).list.sum().alias('months_with_delta_2mo_decrease'),
    pl.col('delta_2mo').list.eval(pl.element() == 0).list.sum().alias('months_with_no_delta_2mo_change'),

    # delta_3mo statistics
    pl.col('delta_3mo').list.mean().round(2).alias('avg_delta_3mo'),
    pl.col('delta_3mo').list.max().round(2).alias('max_delta_3mo_increase'),
    pl.col('delta_3mo').list.min().round(2).alias('max_delta_3mo_decrease'),
    pl.col('delta_3mo').list.eval(pl.element() > 0).list.sum().alias('months_with_delta_3mo_increase'),
    pl.col('delta_3mo').list.eval(pl.element() < 0).list.sum().alias('months_with_delta_3mo_decrease'),
    pl.col('delta_3mo').list.eval(pl.element() == 0).list.sum().alias('months_with_no_delta_3mo_change'),
])

# The following block extracts the last N values from the rolling/statistical lists for each account.
# Each column contains the most recent, second most recent, etc. value from the corresponding list

trend_features = trend_features.with_columns([
    pl.col('avg_2month_rolling_usage_gb').list.get(-(i+1)).round(2).alias(f'last_{i+1}_2mo_rolling_avg')
    for i in range(3)
]).drop('avg_2month_rolling_usage_gb')

trend_features = trend_features.with_columns([
    pl.col('avg_3month_rolling_usage_gb').list.get(-(i+1)).round(2).alias(f'last_{i+1}_3mo_rolling_avg')
    for i in range(2)
]).drop('avg_3month_rolling_usage_gb')

trend_features = trend_features.with_columns([
    pl.col('delta_1mo').list.get(-(i+1)).round(2).alias(f'last_{i+1}_delta_1mo')
    for i in range(3)
]).drop('delta_1mo')

trend_features = trend_features.with_columns([
    pl.col('delta_2mo').list.get(-(i+1)).round(2).alias(f'last_{i+1}_delta_2mo')
    for i in range(2)
]).drop('delta_2mo')

trend_features = trend_features.with_columns([
    pl.col('delta_3mo').list.get(-(i+1)).round(2).alias(f'last_{i+1}_delta_3mo')
    for i in range(1)
]).drop('delta_3mo')

trend_features = trend_features.with_columns([
    pl.col('std_2month_rolling_usage_gb').list.get(-(i+1)).round(2).alias(f'last_{i+1}_2mo_rolling_stdev')
    for i in range(3)
]).drop('std_2month_rolling_usage_gb')

CPU times: user 2.83 s, sys: 13.7 ms, total: 2.84 s
Wall time: 1.87 s


In [14]:
trend_features

rating_account_id,avg_delta_1mo,delta_1mo_volatility,max_delta_1mo_increase,max_delta_1mo_decrease,months_with_delta_1mo_increase,months_with_delta_1mo_decrease,months_with_no_delta_1mo_change,avg_delta_2mo,delta_2mo_volatility,max_delta_2mo_increase,max_delta_2mo_decrease,months_with_delta_2mo_increase,months_with_delta_2mo_decrease,months_with_no_delta_2mo_change,avg_delta_3mo,max_delta_3mo_increase,max_delta_3mo_decrease,months_with_delta_3mo_increase,months_with_delta_3mo_decrease,months_with_no_delta_3mo_change,last_1_2mo_rolling_avg,last_2_2mo_rolling_avg,last_3_2mo_rolling_avg,last_1_3mo_rolling_avg,last_2_3mo_rolling_avg,last_1_delta_1mo,last_2_delta_1mo,last_3_delta_1mo,last_1_delta_2mo,last_2_delta_2mo,last_1_delta_3mo,last_1_2mo_rolling_stdev,last_2_2mo_rolling_stdev,last_3_2mo_rolling_stdev
str,f64,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,u32,u32,u32,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""100010""",0,0.46,0.5,-0.4,1,2,0,-0.2,0.42,0.1,-0.5,1,1,0,0,0,0,0,0,1,0.65,0.6,0.85,0.7,0.7,0.5,-0.4,-0.1,0.1,-0.5,0,0.35,0.28,0.07
"""100017""",-0.13,0.4,0.3,-0.5,1,2,0,-0.05,0.21,0.1,-0.2,1,1,0,-0.4,-0.4,-0.4,0,1,0,0.55,0.65,0.6,0.53,0.67,-0.5,0.3,-0.2,-0.2,0.1,-0.4,0.35,0.21,0.14
"""100036""",0.03,0.75,0.8,-0.7,1,1,1,0.05,1.06,0.8,-0.7,1,1,0,0.1,0.1,0.1,1,0,0,0.6,0.2,0.55,0.47,0.43,0.8,0,-0.7,0.8,-0.7,0.1,0.57,0,0.49
"""100047""",5.03,16.08,15.3,-13.5,2,1,0,0.8,1.41,1.8,-0.2,1,1,0,15.1,15.1,15.1,1,0,0,43.05,42.15,42.25,45,39.97,15.3,-13.5,13.3,1.8,-0.2,15.1,10.82,9.55,9.4
"""100064""",0.03,0.45,0.5,-0.4,1,1,1,-0.15,0.35,0.1,-0.4,1,1,0,0.1,0.1,0.1,1,0,0,0.55,0.5,0.7,0.6,0.57,0.5,-0.4,0,0.1,-0.4,0.1,0.35,0.28,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""999922""",-0.07,1.99,2.2,-1.5,1,2,0,1,0.42,1.3,0.7,2,0,0,-0.2,-0.2,-0.2,0,1,0,2.85,2.5,1.85,2.37,2.43,-1.5,2.2,-0.9,0.7,1.3,-0.2,1.06,1.56,0.64
"""999934""",-2.1,5.11,1,-8,2,1,0,-2.8,6.36,1.7,-7.3,1,1,0,-6.3,-6.3,-6.3,0,1,0,10.5,14.15,13.3,11.6,13.7,-8,0.7,1,-7.3,1.7,-6.3,5.66,0.49,0.71
"""999940""",1.53,7.6,9,-6.2,2,1,0,-0.8,5.09,2.8,-4.4,1,1,0,4.6,4.6,4.6,1,0,0,9.9,8.5,10.7,10.47,8.93,9,-6.2,1.8,2.8,-4.4,4.6,6.36,4.38,1.27


In [15]:
usage_features = aggregated_features.join(
    trend_features, 
    on='rating_account_id', 
    how='left'
).join(
    month_usage,
    on='rating_account_id',
    how='left'
)

In [16]:
usage_features

rating_account_id,avg_monthly_usage_gb,median_monthly_usage_gb,total_usage_gb,usage_std_gb,min_monthly_usage_gb,max_monthly_usage_gb,usage_q25_gb,usage_q75_gb,months_with_roaming,ever_used_roaming,always_used_roaming,zero_usage_months,active_usage_months,avg_delta_1mo,delta_1mo_volatility,max_delta_1mo_increase,max_delta_1mo_decrease,months_with_delta_1mo_increase,months_with_delta_1mo_decrease,months_with_no_delta_1mo_change,avg_delta_2mo,delta_2mo_volatility,max_delta_2mo_increase,max_delta_2mo_decrease,months_with_delta_2mo_increase,months_with_delta_2mo_decrease,months_with_no_delta_2mo_change,avg_delta_3mo,max_delta_3mo_increase,max_delta_3mo_decrease,months_with_delta_3mo_increase,months_with_delta_3mo_decrease,months_with_no_delta_3mo_change,last_1_2mo_rolling_avg,last_2_2mo_rolling_avg,last_3_2mo_rolling_avg,last_1_3mo_rolling_avg,last_2_3mo_rolling_avg,last_1_delta_1mo,last_2_delta_1mo,last_3_delta_1mo,last_1_delta_2mo,last_2_delta_2mo,last_1_delta_3mo,last_1_2mo_rolling_stdev,last_2_2mo_rolling_stdev,last_3_2mo_rolling_stdev,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb
str,f64,f64,f64,f64,f64,f64,f64,f64,u32,bool,bool,u32,u32,f64,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,u32,u32,u32,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""100010""",0.75,0.85,3,0.24,0.4,0.9,0.8,0.9,0,false,false,0,4,0,0.46,0.5,-0.4,1,2,0,-0.2,0.42,0.1,-0.5,1,1,0,0,0,0,0,0,1,0.65,0.6,0.85,0.7,0.7,0.5,-0.4,-0.1,0.1,-0.5,0,0.35,0.28,0.07,0.9,0.8,0.4,0.9
"""100017""",0.57,0.6,2.3,0.22,0.3,0.8,0.5,0.7,2,true,false,0,4,-0.13,0.4,0.3,-0.5,1,2,0,-0.05,0.21,0.1,-0.2,1,1,0,-0.4,-0.4,-0.4,0,1,0,0.55,0.65,0.6,0.53,0.67,-0.5,0.3,-0.2,-0.2,0.1,-0.4,0.35,0.21,0.14,0.7,0.5,0.8,0.3
"""100036""",0.57,0.55,2.3,0.43,0.2,1,0.2,0.9,1,true,false,0,4,0.03,0.75,0.8,-0.7,1,1,1,0.05,1.06,0.8,-0.7,1,1,0,0.1,0.1,0.1,1,0,0,0.6,0.2,0.55,0.47,0.43,0.8,0,-0.7,0.8,-0.7,0.1,0.57,0,0.49,0.9,0.2,0.2,1
"""100047""",42.65,42.25,170.6,8.29,35.4,50.7,35.6,48.9,0,false,false,0,4,5.03,16.08,15.3,-13.5,2,1,0,0.8,1.41,1.8,-0.2,1,1,0,15.1,15.1,15.1,1,0,0,43.05,42.15,42.25,45,39.97,15.3,-13.5,13.3,1.8,-0.2,15.1,10.82,9.55,9.4,35.6,48.9,35.4,50.7
"""100064""",0.62,0.7,2.5,0.22,0.3,0.8,0.7,0.7,1,true,false,0,4,0.03,0.45,0.5,-0.4,1,1,1,-0.15,0.35,0.1,-0.4,1,1,0,0.1,0.1,0.1,1,0,0,0.55,0.5,0.7,0.6,0.57,0.5,-0.4,0,0.1,-0.4,0.1,0.35,0.28,0,0.7,0.7,0.3,0.8
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""999922""",2.35,2.2,9.4,0.92,1.4,3.6,2.1,2.3,0,false,false,0,4,-0.07,1.99,2.2,-1.5,1,2,0,1,0.42,1.3,0.7,2,0,0,-0.2,-0.2,-0.2,0,1,0,2.85,2.5,1.85,2.37,2.43,-1.5,2.2,-0.9,0.7,1.3,-0.2,1.06,1.56,0.64,2.3,1.4,3.6,2.1
"""999934""",11.9,13.3,47.6,3.67,6.5,14.5,12.8,13.8,3,true,false,0,4,-2.1,5.11,1,-8,2,1,0,-2.8,6.36,1.7,-7.3,1,1,0,-6.3,-6.3,-6.3,0,1,0,10.5,14.15,13.3,11.6,13.7,-8,0.7,1,-7.3,1.7,-6.3,5.66,0.49,0.71,12.8,13.8,14.5,6.5
"""999940""",10.3,10.7,41.2,3.78,5.4,14.4,9.8,11.6,1,true,false,0,4,1.53,7.6,9,-6.2,2,1,0,-0.8,5.09,2.8,-4.4,1,1,0,4.6,4.6,4.6,1,0,0,9.9,8.5,10.7,10.47,8.93,9,-6.2,1.8,2.8,-4.4,4.6,6.36,4.38,1.27,9.8,11.6,5.4,14.4


In [17]:
usage_features.shape

(100000, 52)

---

## `customer_interactions`

In [18]:
%%time

interactions_features = customer_interactions.pivot(
    index='customer_id',
    on='type_subtype', 
    values=['n', 'days_since_last'],
    aggregate_function='first' # There is only one value per customer
)

CPU times: user 29.4 ms, sys: 5.88 ms, total: 35.3 ms
Wall time: 69 ms


In [19]:
interactions_features.shape

(42095, 9)

---

## `combined_features`

In [20]:
features = core_data.join(
    usage_features,
    on='rating_account_id',
    how='left'
).join(
    interactions_features,
    on='customer_id',
    how='left'
)

In [21]:
features.head()

rating_account_id,customer_id,age,contract_lifetime_days,remaining_binding_days,has_special_offer,is_magenta1_customer,available_gb,gross_mrc,has_done_upselling,contract_binding_days,completion_rate,is_bounded,is_huawei,is_oneplus,is_samsung,is_xiaomi,is_iphone,n_contracts_per_customer,avg_monthly_usage_gb,median_monthly_usage_gb,total_usage_gb,usage_std_gb,min_monthly_usage_gb,max_monthly_usage_gb,usage_q25_gb,usage_q75_gb,months_with_roaming,ever_used_roaming,always_used_roaming,zero_usage_months,active_usage_months,avg_delta_1mo,delta_1mo_volatility,max_delta_1mo_increase,max_delta_1mo_decrease,months_with_delta_1mo_increase,…,max_delta_2mo_increase,max_delta_2mo_decrease,months_with_delta_2mo_increase,months_with_delta_2mo_decrease,months_with_no_delta_2mo_change,avg_delta_3mo,max_delta_3mo_increase,max_delta_3mo_decrease,months_with_delta_3mo_increase,months_with_delta_3mo_decrease,months_with_no_delta_3mo_change,last_1_2mo_rolling_avg,last_2_2mo_rolling_avg,last_3_2mo_rolling_avg,last_1_3mo_rolling_avg,last_2_3mo_rolling_avg,last_1_delta_1mo,last_2_delta_1mo,last_3_delta_1mo,last_1_delta_2mo,last_2_delta_2mo,last_1_delta_3mo,last_1_2mo_rolling_stdev,last_2_2mo_rolling_stdev,last_3_2mo_rolling_stdev,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb,n_rechnungsanfragen,n_produkte&services-tarifdetails,n_prolongation,n_produkte&services-tarifwechsel,days_since_last_rechnungsanfragen,days_since_last_produkte&services-tarifdetails,days_since_last_prolongation,days_since_last_produkte&services-tarifwechsel
str,str,i64,i64,i64,bool,bool,i64,f64,bool,i64,f64,bool,bool,bool,bool,bool,bool,u32,f64,f64,f64,f64,f64,f64,f64,f64,u32,bool,bool,u32,u32,f64,f64,f64,f64,u32,…,f64,f64,u32,u32,u32,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64
"""289094""","""4.161115""",36,878,325,false,false,20,70,false,1203,0.73,true,false,false,false,false,true,1,0.28,0.15,1.1,0.36,0,0.8,0.1,0.2,1,true,false,1,3,-0.27,0.29,-0.1,-0.6,0,…,-0.2,-0.7,0,2,0,-0.8,-0.8,-0.8,0,1,0,0.05,0.15,0.5,0.1,0.37,-0.1,-0.1,-0.6,-0.2,-0.7,-0.8,0.07,0.07,0.42,0.8,0.2,0.1,0,null,null,null,null,null,null,null,null
"""677626""","""2.429976""",34,998,614,false,false,0,5,false,1612,0.62,true,false,false,true,false,false,1,0.65,0.65,2.6,0.31,0.3,1,0.5,0.8,1,true,false,0,4,0.07,0.55,0.7,-0.3,1,…,0.5,-0.5,1,1,0,0.2,0.2,0.2,1,0,0,0.65,0.4,0.65,0.6,0.53,0.7,-0.2,-0.3,0.5,-0.5,0.2,0.49,0.14,0.21,0.8,0.5,0.3,1,null,null,1,1,null,null,87,118
"""769928""","""3.875044""",36,37,-26,false,true,50,16.94,false,11,3.36,false,false,false,true,false,false,2,0.6,0.55,2.4,0.32,0.3,1,0.4,0.7,0,false,false,0,4,-0.2,0.56,0.4,-0.7,1,…,0.1,-0.3,1,1,0,-0.6,-0.6,-0.6,0,1,0,0.55,0.5,0.65,0.47,0.67,-0.3,0.4,-0.7,0.1,-0.3,-0.6,0.21,0.28,0.49,1,0.3,0.7,0.4,null,null,null,null,null,null,null,null
"""873260""","""4.649933""",50,503,-149,false,true,20,30.2,true,354,1.42,false,false,false,false,false,true,1,0.38,0.25,1.5,0.36,0.1,0.9,0.2,0.3,0,false,false,0,4,0.03,0.67,0.6,-0.7,2,…,0.8,-0.1,1,1,0,0.1,0.1,0.1,1,0,0,0.55,0.6,0.2,0.47,0.43,-0.7,0.6,0.2,-0.1,0.8,0.1,0.49,0.42,0.14,0.1,0.3,0.9,0.2,null,null,null,null,null,null,null,null
"""109774""","""3.851059""",47,331,-328,true,true,null,46.12,false,3,110.33,false,false,false,true,false,false,3,0.35,0.35,1.4,0.29,0.1,0.6,0.1,0.6,1,true,false,0,4,0,0.5,0.5,-0.5,1,…,0.5,-0.5,1,1,0,0,0,0,0,0,1,0.35,0.1,0.35,0.27,0.27,0.5,0,-0.5,0.5,-0.5,0,0.35,0,0.35,0.6,0.1,0.1,0.6,null,null,null,null,null,null,null,null


In [22]:
%%time

# Filling null values from intereactions features
features = features.with_columns([
    pl.when(pl.col(col).is_null())
    .then(0)
    .otherwise(pl.col(col))
    .alias(col)
    for col in features.columns if col.startswith('n_')
] + [
    pl.when(pl.col(col).is_null())
    .then(-1)
    .otherwise(pl.col(col))
    .alias(col)
    for col in features.columns if col.startswith('days_since_last')
])

CPU times: user 2.99 ms, sys: 1.04 ms, total: 4.03 ms
Wall time: 8.44 ms


### Dealing with null values in `available_gb`

In [23]:
features.filter(pl.col('available_gb').is_null()).select('rating_account_id', 'available_gb', 'last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb')

rating_account_id,available_gb,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb
str,i64,f64,f64,f64,f64
"""109774""",null,0.6,0.1,0.1,0.6
"""781755""",null,0.6,0.8,0.3,0
"""827238""",null,0,0.7,0.9,0.1
"""330581""",null,0,0.2,0.2,0.9
"""416121""",null,0.8,0.8,0.3,0.5
…,…,…,…,…,…
"""662172""",null,54.7,62.5,24.7,37.4
"""556788""",null,19.6,32.6,31,65.2
"""283647""",null,51.3,18.1,53.3,39.2


In [24]:
%%time

# Option A1: predict the value with a linear model

# Prepare the data: drop rows with null available_gb and select numeric features
df = (
        features
            .filter(pl.col('available_gb').is_not_null())
            .select('available_gb', 'last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb')
)

X = df[['last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb']]
y = df['available_gb']

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Ridge regression (L2)
ridge = Ridge()
ridge.fit(X_train, y_train)

# Predict and evaluate
y_pred = ridge.predict(X_test)
mae_linear_model = mean_absolute_error(y_test, y_pred)
print(f'MAE Ridge: {mae_linear_model:.4f}')

MAE Ridge: 15.0033
CPU times: user 29.2 ms, sys: 16.2 ms, total: 45.4 ms
Wall time: 84.8 ms


In [25]:
%%time

# Option A2: predict the value with a linear model

# Prepare the data: drop rows with null available_gb and select numeric features
df = (
        features
            .filter(pl.col('available_gb').is_not_null())
            .select('available_gb', 'last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb')
)

X = df[['last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb']]
y = df['available_gb']

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Lasso regression (L1)
lasso = Lasso()
lasso.fit(X_train, y_train)

# Predict and evaluate
y_pred = lasso.predict(X_test)
mae_lasso = mean_absolute_error(y_test, y_pred)
print(f'MAE Lasso: {mae_lasso:.4f}')

MAE Lasso: 14.9996
CPU times: user 59.4 ms, sys: 6.5 ms, total: 65.9 ms
Wall time: 83.2 ms


In [26]:
%%time

# Option B: mean of the previous contract activities

columns_to_average = ['last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb']

# Prepare the data: drop rows with null available_gb and select numeric features
df = (
        features
            .filter(pl.col('available_gb').is_not_null())
            .select('available_gb', 'last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb')
)


# Compute horizontal mean and find closest value
result_df = df.with_columns([
    # Compute horizontal mean
    pl.mean_horizontal(columns_to_average).alias('prediction')
])

mae_horizontal_mean = mean_absolute_error(result_df['available_gb'], result_df['prediction'])
print(f'MAE horizontal_mean: {mae_horizontal_mean:.4f}')

MAE horizontal_mean: 21.8625
CPU times: user 15.3 ms, sys: 2.9 ms, total: 18.2 ms
Wall time: 19.1 ms


The error of the linear models is lower compared to the mean of the activities. **Winning option A2**

In [27]:
# APPLY THE PREDICTION TO MISSING ROWS
# Prepare the data: drop rows with null available_gb and select numeric features
df = (
        features
            .filter(pl.col('available_gb').is_not_null())
            .select('available_gb', 'last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb')
)

X = df.select(['last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb'])
y = df.select('available_gb').to_series()

df_missing = (
        features
            .filter(pl.col('available_gb').is_null())
            .select(pl.exclude("available_gb"))
)

X_missing = df_missing.select(['last_1_month_usage_gb', 'last_2_month_usage_gb', 'last_3_month_usage_gb', 'last_4_month_usage_gb'])


# Fit Lasso regression (L1)
lasso = Lasso()
lasso.fit(X, y)

# Predict
df_missing = df_missing.with_columns(available_gb=lasso.predict(X_missing))

# Since the value cannot be float, mapping the predicted values to similar options from data
available_values = features.filter(pl.col('available_gb').is_not_null()).select(pl.col('available_gb')).unique().to_series().to_list()

def find_closest(val, avail_list):
    'Find closest value from available_values list'
    if val is None:
        return None
    return min(avail_list, key=lambda x: abs(x - val))

# Compute horizontal mean and find closest value
df_missing = df_missing.with_columns([
    # Find closest available value
    pl.col('available_gb').map_elements(
        lambda x: find_closest(x, available_values), 
        return_dtype=pl.Int64
    ).alias('available_gb')
])

In [28]:
%%time

features = pl.concat(
    [
        features.filter(pl.col('available_gb').is_not_null()),
        df_missing
    ],
    how='diagonal'
)

features.head()

CPU times: user 10.1 ms, sys: 995 μs, total: 11.1 ms
Wall time: 12.9 ms


rating_account_id,customer_id,age,contract_lifetime_days,remaining_binding_days,has_special_offer,is_magenta1_customer,available_gb,gross_mrc,has_done_upselling,contract_binding_days,completion_rate,is_bounded,is_huawei,is_oneplus,is_samsung,is_xiaomi,is_iphone,n_contracts_per_customer,avg_monthly_usage_gb,median_monthly_usage_gb,total_usage_gb,usage_std_gb,min_monthly_usage_gb,max_monthly_usage_gb,usage_q25_gb,usage_q75_gb,months_with_roaming,ever_used_roaming,always_used_roaming,zero_usage_months,active_usage_months,avg_delta_1mo,delta_1mo_volatility,max_delta_1mo_increase,max_delta_1mo_decrease,months_with_delta_1mo_increase,…,max_delta_2mo_increase,max_delta_2mo_decrease,months_with_delta_2mo_increase,months_with_delta_2mo_decrease,months_with_no_delta_2mo_change,avg_delta_3mo,max_delta_3mo_increase,max_delta_3mo_decrease,months_with_delta_3mo_increase,months_with_delta_3mo_decrease,months_with_no_delta_3mo_change,last_1_2mo_rolling_avg,last_2_2mo_rolling_avg,last_3_2mo_rolling_avg,last_1_3mo_rolling_avg,last_2_3mo_rolling_avg,last_1_delta_1mo,last_2_delta_1mo,last_3_delta_1mo,last_1_delta_2mo,last_2_delta_2mo,last_1_delta_3mo,last_1_2mo_rolling_stdev,last_2_2mo_rolling_stdev,last_3_2mo_rolling_stdev,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb,n_rechnungsanfragen,n_produkte&services-tarifdetails,n_prolongation,n_produkte&services-tarifwechsel,days_since_last_rechnungsanfragen,days_since_last_produkte&services-tarifdetails,days_since_last_prolongation,days_since_last_produkte&services-tarifwechsel
str,str,i64,i64,i64,bool,bool,i64,f64,bool,i64,f64,bool,bool,bool,bool,bool,bool,u32,f64,f64,f64,f64,f64,f64,f64,f64,u32,bool,bool,u32,u32,f64,f64,f64,f64,u32,…,f64,f64,u32,u32,u32,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64
"""289094""","""4.161115""",36,878,325,false,false,20,70,false,1203,0.73,true,false,false,false,false,true,1,0.28,0.15,1.1,0.36,0,0.8,0.1,0.2,1,true,false,1,3,-0.27,0.29,-0.1,-0.6,0,…,-0.2,-0.7,0,2,0,-0.8,-0.8,-0.8,0,1,0,0.05,0.15,0.5,0.1,0.37,-0.1,-0.1,-0.6,-0.2,-0.7,-0.8,0.07,0.07,0.42,0.8,0.2,0.1,0,0,0,0,0,-1,-1,-1,-1
"""677626""","""2.429976""",34,998,614,false,false,0,5,false,1612,0.62,true,false,false,true,false,false,1,0.65,0.65,2.6,0.31,0.3,1,0.5,0.8,1,true,false,0,4,0.07,0.55,0.7,-0.3,1,…,0.5,-0.5,1,1,0,0.2,0.2,0.2,1,0,0,0.65,0.4,0.65,0.6,0.53,0.7,-0.2,-0.3,0.5,-0.5,0.2,0.49,0.14,0.21,0.8,0.5,0.3,1,0,0,1,1,-1,-1,87,118
"""769928""","""3.875044""",36,37,-26,false,true,50,16.94,false,11,3.36,false,false,false,true,false,false,2,0.6,0.55,2.4,0.32,0.3,1,0.4,0.7,0,false,false,0,4,-0.2,0.56,0.4,-0.7,1,…,0.1,-0.3,1,1,0,-0.6,-0.6,-0.6,0,1,0,0.55,0.5,0.65,0.47,0.67,-0.3,0.4,-0.7,0.1,-0.3,-0.6,0.21,0.28,0.49,1,0.3,0.7,0.4,0,0,0,0,-1,-1,-1,-1
"""873260""","""4.649933""",50,503,-149,false,true,20,30.2,true,354,1.42,false,false,false,false,false,true,1,0.38,0.25,1.5,0.36,0.1,0.9,0.2,0.3,0,false,false,0,4,0.03,0.67,0.6,-0.7,2,…,0.8,-0.1,1,1,0,0.1,0.1,0.1,1,0,0,0.55,0.6,0.2,0.47,0.43,-0.7,0.6,0.2,-0.1,0.8,0.1,0.49,0.42,0.14,0.1,0.3,0.9,0.2,0,0,0,0,-1,-1,-1,-1
"""692379""","""4.382165""",46,80,-25,false,true,40,60.71,false,55,1.45,false,true,false,false,false,false,1,0.55,0.6,2.2,0.3,0.2,0.8,0.4,0.8,2,true,false,0,4,0,0.53,0.6,-0.4,1,…,0.4,-0.6,1,1,0,0,0,0,0,0,1,0.5,0.3,0.6,0.47,0.47,0.6,-0.2,-0.4,0.4,-0.6,0,0.42,0.14,0.28,0.8,0.4,0.2,0.8,0,0,0,0,-1,-1,-1,-1


In [29]:
# Checking if there are other columns to fill

null_counts_features = features.select([
    pl.col(col).is_null().sum().alias(f'{col}_nulls') for col in features.columns
])

null_counts_features.select([
    col for col in null_counts_features.columns if null_counts_features[0, col] != 0
])

shape: (0, 0)
┌┐
╞╡
└┘

### Computing additional features

Computing features based on `available_gb`, since there are no missing values

In [30]:
%%time

# Compute how many times the contract required more than available data, baed on definde threshold of the current plan.

# Compute 20%, 50%, and 80% for available_gb per rating_account_id
thresholds_available_gb = features.group_by('rating_account_id').agg([
    (pl.col('available_gb') / 100 * 25).get(0).round(2).alias('p25'),
    (pl.col('available_gb') / 100 * 50).get(0).round(2).alias('p50'),
    (pl.col('available_gb') / 100 * 70).get(0).round(2).alias('p75'),
])

# Compute, for each month, in which range the usage falls
percentile_exprs = []
for i in range(1, 5):
    percentile_expr = (
        pl.when(pl.col(f'last_{i}_month_usage_gb').is_between(-1, pl.col('p25'), closed='right'))  # -1 otherwise 0 is not counted
        .then(pl.lit('P1'))
        .when(pl.col(f'last_{i}_month_usage_gb').is_between(pl.col('p25'), pl.col('p50'), closed='right'))
        .then(pl.lit('P2'))
        .when(pl.col(f'last_{i}_month_usage_gb').is_between(pl.col('p50'), pl.col('p75'), closed='right'))
        .then(pl.lit('P3'))
        .when(pl.col(f'last_{i}_month_usage_gb').is_between(pl.col('p75'), pl.col('available_gb'), closed='right'))
        .then(pl.lit('P4'))
        .when(pl.col(f'last_{i}_month_usage_gb') > pl.col('available_gb'))
        .then(pl.lit('P5'))  # how many times has exceeded the available data
        .otherwise(pl.lit(None))
        .alias(f'month_{i}_threshold')
    )
    percentile_exprs.append(percentile_expr)

# Compute how many times, in the past 4 months, the usage felt in specific ranges
count_exprs = []
for p in range(1, 6):
    count_expr = sum(
        (pl.col(f'month_{i}_threshold') == f'P{p}').cast(pl.Int32)
        for i in range(1, 5)
    ).alias(f'times_in_p{p}')
    count_exprs.append(count_expr)

# Final computation
features = (
    features
    .join(
        thresholds_available_gb,
        on='rating_account_id',
        how='left')
    .with_columns(percentile_exprs)
    .with_columns(count_exprs)
).drop(['p25', 'p50', 'p75', 'month_1_threshold', 'month_2_threshold', 'month_3_threshold', 'month_4_threshold'])

CPU times: user 66.6 ms, sys: 16.5 ms, total: 83 ms
Wall time: 119 ms


In [31]:
# Check that the sum of times_in_p1 to times_in_p5 is always 4 (number of billing months) for each row
check_sum = features.select(
    (pl.col('times_in_p1') + pl.col('times_in_p2') + pl.col('times_in_p3') + pl.col('times_in_p4') + pl.col('times_in_p5')).alias('sum_p')
)

# Count how many rows do not have sum == 4
invalid_rows = check_sum.filter(pl.col('sum_p') != 4).height
print(f"Number of invalid rows: {invalid_rows}")

Number of invalid rows: 0


---

## `final features dataframe`

In [32]:
features.shape

(100000, 83)

In [33]:
features.head()

rating_account_id,customer_id,age,contract_lifetime_days,remaining_binding_days,has_special_offer,is_magenta1_customer,available_gb,gross_mrc,has_done_upselling,contract_binding_days,completion_rate,is_bounded,is_huawei,is_oneplus,is_samsung,is_xiaomi,is_iphone,n_contracts_per_customer,avg_monthly_usage_gb,median_monthly_usage_gb,total_usage_gb,usage_std_gb,min_monthly_usage_gb,max_monthly_usage_gb,usage_q25_gb,usage_q75_gb,months_with_roaming,ever_used_roaming,always_used_roaming,zero_usage_months,active_usage_months,avg_delta_1mo,delta_1mo_volatility,max_delta_1mo_increase,max_delta_1mo_decrease,months_with_delta_1mo_increase,…,avg_delta_3mo,max_delta_3mo_increase,max_delta_3mo_decrease,months_with_delta_3mo_increase,months_with_delta_3mo_decrease,months_with_no_delta_3mo_change,last_1_2mo_rolling_avg,last_2_2mo_rolling_avg,last_3_2mo_rolling_avg,last_1_3mo_rolling_avg,last_2_3mo_rolling_avg,last_1_delta_1mo,last_2_delta_1mo,last_3_delta_1mo,last_1_delta_2mo,last_2_delta_2mo,last_1_delta_3mo,last_1_2mo_rolling_stdev,last_2_2mo_rolling_stdev,last_3_2mo_rolling_stdev,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb,n_rechnungsanfragen,n_produkte&services-tarifdetails,n_prolongation,n_produkte&services-tarifwechsel,days_since_last_rechnungsanfragen,days_since_last_produkte&services-tarifdetails,days_since_last_prolongation,days_since_last_produkte&services-tarifwechsel,times_in_p1,times_in_p2,times_in_p3,times_in_p4,times_in_p5
str,str,i64,i64,i64,bool,bool,i64,f64,bool,i64,f64,bool,bool,bool,bool,bool,bool,u32,f64,f64,f64,f64,f64,f64,f64,f64,u32,bool,bool,u32,u32,f64,f64,f64,f64,u32,…,f64,f64,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i32,i32,i32,i32,i32
"""289094""","""4.161115""",36,878,325,false,false,20,70,false,1203,0.73,true,false,false,false,false,true,1,0.28,0.15,1.1,0.36,0,0.8,0.1,0.2,1,true,false,1,3,-0.27,0.29,-0.1,-0.6,0,…,-0.8,-0.8,-0.8,0,1,0,0.05,0.15,0.5,0.1,0.37,-0.1,-0.1,-0.6,-0.2,-0.7,-0.8,0.07,0.07,0.42,0.8,0.2,0.1,0,0,0,0,0,-1,-1,-1,-1,4,0,0,0,0
"""677626""","""2.429976""",34,998,614,false,false,0,5,false,1612,0.62,true,false,false,true,false,false,1,0.65,0.65,2.6,0.31,0.3,1,0.5,0.8,1,true,false,0,4,0.07,0.55,0.7,-0.3,1,…,0.2,0.2,0.2,1,0,0,0.65,0.4,0.65,0.6,0.53,0.7,-0.2,-0.3,0.5,-0.5,0.2,0.49,0.14,0.21,0.8,0.5,0.3,1,0,0,1,1,-1,-1,87,118,0,0,0,0,4
"""769928""","""3.875044""",36,37,-26,false,true,50,16.94,false,11,3.36,false,false,false,true,false,false,2,0.6,0.55,2.4,0.32,0.3,1,0.4,0.7,0,false,false,0,4,-0.2,0.56,0.4,-0.7,1,…,-0.6,-0.6,-0.6,0,1,0,0.55,0.5,0.65,0.47,0.67,-0.3,0.4,-0.7,0.1,-0.3,-0.6,0.21,0.28,0.49,1,0.3,0.7,0.4,0,0,0,0,-1,-1,-1,-1,4,0,0,0,0
"""873260""","""4.649933""",50,503,-149,false,true,20,30.2,true,354,1.42,false,false,false,false,false,true,1,0.38,0.25,1.5,0.36,0.1,0.9,0.2,0.3,0,false,false,0,4,0.03,0.67,0.6,-0.7,2,…,0.1,0.1,0.1,1,0,0,0.55,0.6,0.2,0.47,0.43,-0.7,0.6,0.2,-0.1,0.8,0.1,0.49,0.42,0.14,0.1,0.3,0.9,0.2,0,0,0,0,-1,-1,-1,-1,4,0,0,0,0
"""692379""","""4.382165""",46,80,-25,false,true,40,60.71,false,55,1.45,false,true,false,false,false,false,1,0.55,0.6,2.2,0.3,0.2,0.8,0.4,0.8,2,true,false,0,4,0,0.53,0.6,-0.4,1,…,0,0,0,0,0,1,0.5,0.3,0.6,0.47,0.47,0.6,-0.2,-0.4,0.4,-0.6,0,0.42,0.14,0.28,0.8,0.4,0.2,0.8,0,0,0,0,-1,-1,-1,-1,4,0,0,0,0


# Features cleaning

## Correlation analysis

In [67]:
numeric_and_booleans_features = features.select(cs.numeric())

In [68]:
%%time

correlation_matrix = numeric_and_booleans_features.corr()

CPU times: user 122 ms, sys: 9.49 ms, total: 132 ms
Wall time: 117 ms


In [69]:
correlation_matrix

age,contract_lifetime_days,remaining_binding_days,available_gb,gross_mrc,contract_binding_days,completion_rate,n_contracts_per_customer,avg_monthly_usage_gb,median_monthly_usage_gb,total_usage_gb,usage_std_gb,min_monthly_usage_gb,max_monthly_usage_gb,usage_q25_gb,usage_q75_gb,months_with_roaming,zero_usage_months,active_usage_months,avg_delta_1mo,delta_1mo_volatility,max_delta_1mo_increase,max_delta_1mo_decrease,months_with_delta_1mo_increase,months_with_delta_1mo_decrease,months_with_no_delta_1mo_change,avg_delta_2mo,delta_2mo_volatility,max_delta_2mo_increase,max_delta_2mo_decrease,months_with_delta_2mo_increase,months_with_delta_2mo_decrease,months_with_no_delta_2mo_change,avg_delta_3mo,max_delta_3mo_increase,max_delta_3mo_decrease,months_with_delta_3mo_increase,months_with_delta_3mo_decrease,months_with_no_delta_3mo_change,last_1_2mo_rolling_avg,last_2_2mo_rolling_avg,last_3_2mo_rolling_avg,last_1_3mo_rolling_avg,last_2_3mo_rolling_avg,last_1_delta_1mo,last_2_delta_1mo,last_3_delta_1mo,last_1_delta_2mo,last_2_delta_2mo,last_1_delta_3mo,last_1_2mo_rolling_stdev,last_2_2mo_rolling_stdev,last_3_2mo_rolling_stdev,last_1_month_usage_gb,last_2_month_usage_gb,last_3_month_usage_gb,last_4_month_usage_gb,n_rechnungsanfragen,n_produkte&services-tarifdetails,n_prolongation,n_produkte&services-tarifwechsel,days_since_last_rechnungsanfragen,days_since_last_produkte&services-tarifdetails,days_since_last_prolongation,days_since_last_produkte&services-tarifwechsel,times_in_p1,times_in_p2,times_in_p3,times_in_p4,times_in_p5
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,-0.0629525879053476,0.007917425991875568,0.0020015308181241475,-0.0014248647679732413,-0.04689189462413721,0.00353602431532941,-0.0033925108458075066,-0.07313075894941969,-0.07117159510219147,-0.07313072441998948,-0.0752667069178047,-0.06588505936769457,-0.07627248976389259,-0.06852675138912509,-0.07211471686542024,0.0024213213370002985,0.010187406343503983,-0.0101874063435039,0.0007023898149709626,-0.0698694468472535,-0.06434228889805703,0.06525763675673277,-0.0027109579596412622,-0.003923417078656138,0.013449505856104643,0.0038771650238448078,-0.05982538993845558,-0.03588147143189146,0.0418930823776148,0.002696058456064011,-0.007157296164963452,0.012284435206751758,0.0007018860866903546,0.0007018860866903546,0.0007018860866903546,-0.0027964731629029245,-0.00016122916901736825,0.008278462206978325,-0.07023640985754045,-0.07118943304127716,-0.07216078833847767,-0.07239798432566195,-0.07251675544933449,-0.0022278083988556082,0.004810231937738154,-0.0018765816242892104,0.002577746601875384,0.0029294087217661374,0.0007018860866903546,-0.05818506841796149,-0.06188408762322952,-0.05992545507252074,-0.06797684008539309,-0.06941992923036817,-0.06614590372886091,-0.067636313903008,-0.0029619271116923386,-0.0036624528884935327,-0.0010602562815131568,0.0024569564673974044,-0.002173552477055877,-0.0017217017286030646,-0.004082781479784842,0.0007222265215127118,0.047705646519236894,0.012602850196615879,-0.011356125642459285,-0.02461459789404415,-0.04884294998721446
-0.06295258790534759,1,-0.0003783910010728704,-0.032126065687675204,0.0030451716731358623,0.8171535046016242,-0.07300261199879411,0.0023650517704186607,0.7632866265204388,0.7508626128730621,0.7632872022690604,0.7272201981256826,0.710920257111116,0.7732354631860344,0.7318593393860774,0.7537621693882587,-0.005097468591353552,-0.09475926466371004,0.09475926466370997,-0.0015123138415764018,0.6811245720092849,0.630724779896917,-0.6291921881077874,0.03264938171411658,0.03404249035232747,-0.1352208870762016,0.0048604813883570365,0.5801060874511151,0.3801496092665654,-0.3739104433991196,0.02229034000445165,0.017848040212429837,-0.1103229664401533,-0.0015110509232823767,-0.0015110509232823767,-0.0015110

In [70]:
fig = px.imshow(
    correlation_matrix.to_numpy(),
    labels=dict(x='Features', y='Features', color='Correlation'),
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1,
    aspect='auto'
)
fig.update_layout(
    width=1800,
    height=1800,
    title='Correlation Matrix Heatmap'
)
fig.update_layout(title='Correlation Matrix Heatmap')
fig.show()

# Blue means that variable X and variable Y follow the same behaviour (both increasing or decreasing)
# Red means that variable X has the opposite behaviour of variable Y

In [71]:
%%time 

highly_correlated_features = (
    correlation_matrix
    .unpivot(index=None, variable_name='col1', value_name='correlation')
    .with_columns([
        pl.repeat(correlation_matrix.columns, correlation_matrix.height).flatten().alias('col2'),
        pl.col('correlation').abs()
    ])
    .filter(pl.col('col1') != pl.col('col2'))
    .filter(pl.col('col1') < pl.col('col2'))  # Keep only one pair per combination
    .with_columns([
        pl.when(pl.col('correlation') > 0.99)
            .then(pl.lit('identical'))
        .when(pl.col('correlation') > 0.8)
            .then(pl.lit('high'))
        .when(pl.col('correlation') > 0.7)
            .then(pl.lit('medium'))
        .otherwise(pl.lit('ok'))
        .alias('analysis')
    ])
    .sort('correlation', descending=True)
)

CPU times: user 2.45 ms, sys: 945 μs, total: 3.39 ms
Wall time: 3.44 ms


In [45]:
highly_correlated_features

col1,correlation,col2,analysis
str,f64,str,str
"""active_usage_months""",1,"""zero_usage_months""","""identical"""
"""avg_delta_3mo""",1,"""max_delta_3mo_increase""","""identical"""
"""avg_delta_3mo""",1,"""max_delta_3mo_decrease""","""identical"""
"""avg_delta_3mo""",1,"""last_1_delta_3mo""","""identical"""
"""max_delta_3mo_decrease""",1,"""max_delta_3mo_increase""","""identical"""
…,…,…,…
"""last_4_month_usage_gb""",0.000030343105303516372,"""n_prolongation""","""ok"""
"""last_1_delta_1mo""",0.000022458070975042498,"""n_rechnungsanfragen""","""ok"""
"""last_2_delta_1mo""",0.000004109764369784097,"""months_with_delta_3mo_decrease""","""ok"""


In [47]:
highly_correlated_features.group_by('analysis', maintain_order=True).len()

analysis,len
str,u32
"""identical""",19
"""high""",158
"""medium""",99
"""ok""",2139


## Variance analysis

In [72]:
only_numeric_features = features.select(cs.numeric() & ~cs.boolean())

In [ ]:
%%time

# Compute the standard deviation
variance_analysis = (
    features
    .select([
        pl.col(col).std()
        for col in numeric_and_booleans_features.columns
    ])
    .unpivot(variable_name='feature', value_name='stdev')
    .sort('stdev', descending=True)
)

CPU times: user 16.2 ms, sys: 878 μs, total: 17.1 ms
Wall time: 21.7 ms


In [104]:
variance_analysis

feature,stdev
str,f64
"""months_with_no_delta_3mo_change""",0.17850662773735168
"""active_usage_months""",0.23464950586759398
"""zero_usage_months""",0.23464950586759403
"""months_with_no_delta_2mo_change""",0.25738762612818217
"""months_with_no_delta_1mo_change""",0.31996979811650533
…,…
"""days_since_last_produkte&services-tarifwechsel""",42.254754177537265
"""total_usage_gb""",69.21684430203868
"""remaining_binding_days""",345.64345852492


In [99]:
%%time

# Compute variance, std, and unique ratio for each numeric/boolean feature
variance_analysis = (
    features
    .select([
        pl.col(col).std().alias('std_dev')
        for col in numeric_and_booleans_features.columns
    ])
)


DuplicateError: the name 'std_dev' is duplicate

It's possible that multiple expressions are returning the same default column name. If this is the case, try renaming the columns with `.alias("new_name")` to avoid duplicate column names.

In [98]:
variance_analysis

<generator object <genexpr> at 0x73d11d5dbca0>

In [92]:
%%time

# Unique count dataframe (excluding boolean columns)
unique_count = (
    features
    .select([
        pl.col(col).n_unique().alias(col)
        for col in only_numeric_features.columns
    ])
    .unpivot(variable_name="feature", value_name="unique_count")
    .sort("unique_count")
)




CPU times: user 99.1 ms, sys: 2.46 ms, total: 102 ms
Wall time: 211 ms


In [93]:
unique_count

feature,unique_count
str,u32
"""months_with_delta_3mo_increase""",2
"""months_with_delta_3mo_decrease""",2
"""months_with_no_delta_3mo_change""",2
"""months_with_delta_2mo_increase""",3
"""months_with_delta_2mo_decrease""",3
…,…
"""completion_rate""",2487
"""contract_binding_days""",2516
"""usage_std_gb""",2671


In [ ]:
# Do not remove this column because they can have only few values by design - based on number of billing months for example
do_not_remove_numeric = [
    'months_with_delta_3mo_increase',
    'months_with_delta_3mo_decrease',
    'months_with_no_delta_3mo_change',
    'months_with_delta_2mo_increase',
    'months_with_delta_2mo_decrease',
    'months_with_no_delta_2mo_change',
    'months_with_delta_1mo_increase',
    'months_with_delta_1mo_decrease',
    'months_with_no_delta_1mo_change',
    'zero_usage_months',
    'active_usage_months',
    'max_delta_1mo_increase',
    'max_delta_1mo_decrease',
    'max_delta_2mo_increase',
    'max_delta_2mo_decrease',
    'max_delta_3mo_increase',
    'max_delta_3mo_decrease',
    'times_in_p1',
    'times_in_p2',
    'times_in_p3',
    'times_in_p4',
    'times_in_p5',
]

do_not_remove_target = ['has_done_upselling']

do_not_remove = do_not_remove_numeric + do_not_remove_target


# Storing